In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.vision import *
from fastai.metrics import error_rate
from ring_doorbell import Ring
from datetime import datetime

import os, sys, glob, shutil
import subprocess
import binascii


In [2]:
myring = Ring('', '')

if myring.is_connected:
  doorbell = myring.doorbells[0]
else:
  exit

#for event in doorbell.history(limit=100, kind='motion'):
#  print("ID:       {0}_{1}.mgp4".format(event['id'], str(datetime.fromisoformat( str(event['created_at']) ).timestamp())[0:-2] ))


In [ ]:
events = []
counter = 0
history = doorbell.history(limit=100)
while (len(history) > 0):
  events += history
  counter += len(history)
  history = doorbell.history(older_than=history[-1]['id'])

motion = len([m['kind'] for m in events if m['kind'] == 'motion'])
ding = len([m['kind'] for m in events if m['kind'] == 'ding'])
on_demand = len([m['kind'] for m in events if m['kind'] == 'on_demand'])

print("\tTotal videos: {}".format(counter))
print("\tDing triggered: {}".format(ding))
print("\tMotion triggered: {}".format(motion))
print("\tOn-Demand triggered: {}".format(on_demand))


In [8]:

for events in doorbell.history(limit=10):
  print("\t{}_{}.mgp4".format(events['id'], str(datetime.fromisoformat( str(events['created_at']) ).timestamp())[0:-2] ))


	6672780415069202652_1553627759.mgp4
	6672777357052487900_1553627047.mgp4
	6672775767914588380_1553626677.mgp4
	6672775059244984540_1553626512.mgp4
	6672774221726361820_1553626317.mgp4
	6672773616135973084_1553626176.mgp4
	6672772224566569180_1553625852.mgp4
	6672771795069839580_1553625752.mgp4
	6672770747097819356_1553625508.mgp4
	6672770455040043228_1553625440.mgp4


In [ ]:
def extract_frame_list(filename, frame_list, saved_directory="saved"):
    frame_list = sorted(frame_list, key=lambda x: x['frame_number'])
    if not os.path.exists(saved_directory):
        os.makedirs(saved_directory)
    frames_used = []
    frames_used.append(frame_list[0]['frame_number'])
    portion = "select=\'eq(n\,{})".format(frame_list[0]['frame_number'])
    for instance in frame_list[1:]:
        while instance['frame_number'] in frames_used:
            instance['frame_number'] += 1
        frame_index = int(instance['frame_number'])
        portion += '+eq(n\,{})'.format(frame_index)
        frames_used.append(frame_index)
    portion += "\'"
    print("portion = ", portion)
    random_string = binascii.b2a_hex(os.urandom(5)).decode()
    command_list =  ["ffmpeg", "-i", filename, "-qscale:v", "1", "-vf", portion, "-vsync", "0", "{}/{}_frames%d.jpg".format(saved_directory, random_string)]
    print(command_list)
    subprocess.call(command_list)
    for i in range(1, len(frame_list)+1):
        os.rename("{}/{}_frames{}.jpg".format(saved_directory, random_string,i), os.path.join(saved_directory, frame_list[i-1]['filename']))
    return len(frame_list)